# Iteration 1: modelling unplanned arrivals

## Plan English Summary

We increased the scope of the model by including a treatment process for unplanned (emergency) arrivals to the model.  Like arrrivals each type of patient has their own treatment distribution for sampling.  The treatment process also introduces a shared resource into the model - critical care beds - and a First come first served queue.

The input command is explicit that the code should be *modified* and that the full code listing should be produced.




## Notebook imports

In [1]:
from IPython.display import Markdown, display

import numpy as np
import math

## Show query 

In [2]:
display(Markdown("inputs/iteration_1.md"))

### Main command

Modify the model to add in an unplanned admissions treatment process.

Show the full code listing.  Do not hide any functions or classes.

### General model logic

The model includes a resource: critical care beds.

After patients have arrived they follow the unplanned admissions process. In the unplanned admissions process patients request and wait for a critical care bed resource to become available in a FIFO queue, and undergo treatment.  Following treatment the patient is recorded as "discharged", but the bed is not released until intensive cleaning has taken place.  

The model should print out useful information after each event.

### Simulation inputs

the length of stay (measured in hours) for treatment of patients in the CCU follow source specific distributions

Accident and Emergency: Lognormal: mean 128.79 and standard deviation 267.51
the Wards = Lognormal: mean 177.89 and standard deviation 276.54
Emergency surgery = Lognormal: mean 140.15 and standard deviation 218.02
other hospitals = Lognormal: mean 212.86 and standard deviation 457.67
the X-Ray department = Lognormal: mean 87.53 and standard deviation 108.67

The mean and standard deviation of the Lognormal distributions must be converted to the mean and standard deviation of the underlying normal distribution.

There are 24 critical care beds in the model.  These are shared across all types of patient.

Intensive cleaning takes a fixed amount of time: 5 hours.




## Date and time of input to LLM

> 1233 27th November 2023 

## Observations on the LLM Output

* The LoS parameters are hardcoded and stored in a list of tuples.  The new function `unplanned_admission` function uses them via lookup.
* Treatment time is sampled from the `numpy.random.lognormal` distribution and converts the mean and standard deviation of the lognormal to the underlying normal as follows:

```python
mu = np.log(stay_distribution[0]) - 0.5 * np.log(1 + (stay_distribution[1] / stay_distribution[0])**2)

sigma = np.sqrt(np.log(1 + (stay_distribution[1] / stay_distribution[0])**2)))
```

* The model included a new `simpy` resource `critical_care_beds`. Harded coded to 24.
* The model code provided runs without raise any runtime errors

### Testing LLM code snippets 

* Test of the lognormal sampling.  The code below is our own verified code. The source is given in the docstring.

In [3]:
class Lognormal:
    """
    Encapsulates a lognormal distirbution

    Created by tester:

    Source: https://tommonks.github.io/treatment-centre-sim/content/02_model_code/04_model.html
    """
    def __init__(self, mean, stdev, random_seed=None):
        """
        Params:
        -------
        mean: float
            mean of the lognormal distribution
            
        stdev: float
            standard dev of the lognormal distribution
            
        random_seed: int, optional (default=None)
            Random seed to control sampling
        """
        self.rng = np.random.default_rng(seed=random_seed)
        mu, sigma = self.normal_moments_from_lognormal(mean, stdev**2)
        self.mu = mu
        self.sigma = sigma
        
    def normal_moments_from_lognormal(self, m, v):
        '''
        Returns mu and sigma of normal distribution
        underlying a lognormal with mean m and variance v
        source: https://blogs.sas.com/content/iml/2014/06/04/simulate-lognormal
        -data-with-specified-mean-and-variance.html

        Params:
        -------
        m: float
            mean of lognormal distribution
        v: float
            variance of lognormal distribution
                
        Returns:
        -------
        (float, float)
        '''
        phi = math.sqrt(v + m**2)
        mu = math.log(m**2/phi)
        sigma = math.sqrt(math.log(phi**2/m**2))
        return mu, sigma
        
    def sample(self):
        """
        Sample from the normal distribution
        """
        return self.rng.lognormal(self.mu, self.sigma)

In [4]:
def llm_lognormal(stay_distribution):
    '''
    Wrapper for LLM lognormal code for calculating mu, sigma of underlying
    normal distribution.

    The function has been created by the tester. The calculation generated
    by the LLM.

    Params:
    ------
    stay_distribution
    '''
    
    mu = np.log(stay_distribution[0]) - 0.5 * np.log(1 + (stay_distribution[1] / stay_distribution[0])**2)

    sigma = np.sqrt(np.log(1 + (stay_distribution[1] / stay_distribution[0])**2))

    return mu, sigma

### Running the tests

We will use actual data from the case study.

In [5]:
mean, std = 128.79, 267.51
expected_moments = Lognormal(mean, std)
print(expected_moments.mu, expected_moments.sigma)

llm_mu, llm_sigma = llm_lognormal((mean, std))
print(llm_mu, llm_sigma)

(llm_mu, llm_sigma) == (expected_moments.mu, expected_moments.sigma)

4.022977459259615 1.2924439729790718
4.022977459259615 1.2924439729790718


True

## Sources reported by the LLM

* https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7491890/
* https://www.sciencedirect.com/science/article/abs/pii/S2211692316301564
* https://journals.sagepub.com/doi/full/10.1177/1460458216628314
* https://github.com/KarrLab/de_sim

## LLM Output

In this modified code, the critical care unit (CCU) model is extended to include an unplanned admissions treatment process. The model now includes a resource for critical care beds, and patients follow the unplanned admissions process, requesting and waiting for a critical care bed resource to become available in a FIFO queue, and undergoing treatment. After treatment, the patient is recorded as "discharged," but the bed is not released until intensive cleaning has taken place. The model prints out useful information after each event.

The length of stay for treatment of patients in the CCU follows source-specific lognormal distributions. The mean and standard deviation of the lognormal distributions are converted to the mean and standard deviation of the underlying normal distribution.

The code creates a class `CCUModel` that defines the CCU model with the unplanned admissions treatment process. It includes generator functions for patient arrival from each source, an unplanned admissions treatment process, and a `run` method to start the simulation. The simulation environment is created using `simpy.Environment()`, and an instance of the `CCUModel` is created to run the simulation.

The code uses the `numpy.random.lognormal` function to sample treatment times from lognormal distributions for patients from each source. After each patient's arrival and treatment, the model prints out useful information including the patient identifier, arrival time, admission to a critical care bed, discharge from the critical care bed, and availability of the critical care bed for the next patient.

The code is based on the requirements specified and uses SimPy 4.1.1 and Python 3.10.4, as requested. For more details on SimPy and Python, please refer to the official documentation and release notes[1][3].

Citations:
[1] https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7491890/

[2] https://www.researchgate.net/publication/344421997_Discrete-Event_Simulation_Modeling_of_Critical_Care_Flow_New_Hospital_Old_Challenges

[3] https://www.sciencedirect.com/science/article/abs/pii/S2211692316301564

[4] https://journals.sagepub.com/doi/full/10.1177/1460458216628314

[5] https://github.com/KarrLab/de_sim

In [6]:
import simpy
import numpy as np

# Define the critical care unit (CCU) model with unplanned admissions treatment process
class CCUModel:
    def __init__(self, env, run_length=12*30*24):  # Default run length is 12 months
        self.env = env
        self.run_length = run_length
        self.sources = ['Accident and Emergency', 'the Wards', 'Emergency surgery', 'other hospitals', 'the X-Ray department']
        self.interarrival_distributions = [22.72, 26.0, 37.0, 47.2, 575.0]
        self.stay_distributions = [(128.79, 267.51), (177.89, 276.54), (140.15, 218.02), (212.86, 457.67), (87.53, 108.67)]
        self.patient_count = 0
        self.critical_care_beds = simpy.Resource(env, capacity=24)

    # Generator function for patient arrival from Accident and Emergency
    def patient_arrival_AE(self):
        while True:
            yield self.env.timeout(np.random.exponential(22.72))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from Accident and Emergency at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.stay_distributions[0]))

    # Generator function for patient arrival from the Wards
    def patient_arrival_wards(self):
        while True:
            yield self.env.timeout(np.random.exponential(26.0))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from the Wards at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.stay_distributions[1]))

    # Generator function for patient arrival from Emergency surgery
    def patient_arrival_surgery(self):
        while True:
            yield self.env.timeout(np.random.exponential(37.0))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from Emergency surgery at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.stay_distributions[2]))

    # Generator function for patient arrival from other hospitals
    def patient_arrival_other_hospitals(self):
        while True:
            yield self.env.timeout(np.random.exponential(47.2))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from other hospitals at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.stay_distributions[3]))

    # Generator function for patient arrival from the X-Ray department
    def patient_arrival_X_ray(self):
        while True:
            yield self.env.timeout(np.random.exponential(575.0))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from the X-Ray department at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.stay_distributions[4]))

    # Unplanned admissions treatment process
    def unplanned_admission(self, stay_distribution):
        with self.critical_care_beds.request() as req:
            yield req
            print(f"Patient {self.patient_count} admitted to critical care bed at time {self.env.now}")
            treatment_time = np.random.lognormal(np.log(stay_distribution[0]) - 0.5 * np.log(1 + (stay_distribution[1] / stay_distribution[0])**2),
                                                 np.sqrt(np.log(1 + (stay_distribution[1] / stay_distribution[0])**2)))
            yield self.env.timeout(treatment_time)
            print(f"Patient {self.patient_count} discharged from critical care bed at time {self.env.now}")
            yield self.env.timeout(5)  # Intensive cleaning time
            print(f"Critical care bed is available for next patient at time {self.env.now}")

    # Run the simulation
    def run(self):
        env.process(self.patient_arrival_AE())
        env.process(self.patient_arrival_wards())
        env.process(self.patient_arrival_surgery())
        env.process(self.patient_arrival_other_hospitals())
        env.process(self.patient_arrival_X_ray())
        env.run(until=self.run_length)

# Create a SimPy environment
env = simpy.Environment()

# Create an instance of the CCU model and run the simulation
ccu_model = CCUModel(env)
ccu_model.run()


Patient 1 arrived from Accident and Emergency at time 4.347456924014187
Patient 1 admitted to critical care bed at time 4.347456924014187
Patient 2 arrived from Emergency surgery at time 9.962989406369571
Patient 2 admitted to critical care bed at time 9.962989406369571
Patient 3 arrived from Emergency surgery at time 19.5816128798615
Patient 3 admitted to critical care bed at time 19.5816128798615
Patient 3 discharged from critical care bed at time 20.351426976973627
Critical care bed is available for next patient at time 25.351426976973627
Patient 4 arrived from Emergency surgery at time 35.77357762400401
Patient 4 admitted to critical care bed at time 35.77357762400401
Patient 4 discharged from critical care bed at time 35.8534853478392
Patient 5 arrived from other hospitals at time 37.44460587267154
Patient 5 admitted to critical care bed at time 37.44460587267154
Patient 6 arrived from Accident and Emergency at time 39.01109128278482
Patient 6 admitted to critical care bed at time

## Modifications to LLM code to test for correct model outputs.

* Extreme value test 1: `critical_care_beds` = 1
    * Expected result: queues form after first arrival.
    * Actual result (PASS): queues form after first arrival. 
* Extreme value test 2: means of arrival_distributions set to $M$ a very large number.
    * Expected result: after 24 arrivals queues form and no patients are admitted.
    * Actual result (PASS): after 24 arrivals queues form and no patients are admitted.

### Extreme value test 1

PASS.

In [7]:
env = simpy.Environment()

# Create an instance of the CCU model and run the simulation
ccu_model = CCUModel(env)
ccu_model.critical_care_beds = simpy.Resource(env, capacity=1) 
ccu_model.run()

Patient 1 arrived from the Wards at time 8.59081113528799
Patient 1 admitted to critical care bed at time 8.59081113528799
Patient 2 arrived from Accident and Emergency at time 15.417093375895417
Patient 3 arrived from the Wards at time 21.733445913529966
Patient 4 arrived from Accident and Emergency at time 27.532861840267476
Patient 5 arrived from Emergency surgery at time 28.6314011623859
Patient 6 arrived from Accident and Emergency at time 30.54071272758502
Patient 7 arrived from other hospitals at time 32.054697713469444
Patient 8 arrived from the Wards at time 36.76306512336652
Patient 9 arrived from Accident and Emergency at time 37.0183778491667
Patient 10 arrived from the Wards at time 43.677796919972984
Patient 11 arrived from Emergency surgery at time 45.01845494563429
Patient 12 arrived from other hospitals at time 55.893702090063336
Patient 13 arrived from Accident and Emergency at time 80.91785144808935
Patient 14 arrived from Emergency surgery at time 81.87672355070947


## Extreme value test 2

PASS

In [8]:
M = 10_000_000

env = simpy.Environment()

# Create an instance of the CCU model and run the simulation
ccu_model = CCUModel(env)

ccu_model.stay_distributions = [(M, 267.51), 
                                (M, 276.54), 
                                (M, 218.02), 
                                (M, 457.67), 
                                (M, 108.67)]
ccu_model.run()

Patient 1 arrived from Emergency surgery at time 35.325979250354806
Patient 1 admitted to critical care bed at time 35.325979250354806
Patient 2 arrived from Accident and Emergency at time 36.13502926031134
Patient 2 admitted to critical care bed at time 36.13502926031134
Patient 3 arrived from Accident and Emergency at time 77.39645274757837
Patient 3 admitted to critical care bed at time 77.39645274757837
Patient 4 arrived from Accident and Emergency at time 80.28752125232296
Patient 4 admitted to critical care bed at time 80.28752125232296
Patient 5 arrived from the Wards at time 94.34145606231476
Patient 5 admitted to critical care bed at time 94.34145606231476
Patient 6 arrived from the Wards at time 103.02686464891649
Patient 6 admitted to critical care bed at time 103.02686464891649
Patient 7 arrived from other hospitals at time 116.021729798091
Patient 7 admitted to critical care bed at time 116.021729798091
Patient 8 arrived from Accident and Emergency at time 123.420179663114